# Base parameters and combinations with QR

In the following, the QR decomposition is used to find columns in a matrix (e.g. dynamics regressor) that are independent and therefore are a choice of a basis for the identifiable subspace. It is then determined which linear combinations of the other dependent columns are forming the base columns/parameters in each of those. This is also necessary for the std essential parameters.

In [2]:
import numpy as np; np.set_printoptions(formatter={'float': '{: 0.2f}'.format})

In [3]:
a = np.array([[1.0, 2.0,  .0,  .0, 2.0],
              [1.0, 2.0,  .0,  .0, 1.0],
              [ .0,  .0, 1.0, 5.0,  .0],
              [ .0,  .0, 1.0, 5.0,  .0],
              [2.0, 4.0,  .0,  .0,  .0],
              [ .0,  .0, 0.5, 2.5,  .0]])
print(a.shape)
r = np.linalg.matrix_rank(a)
r

(6, 5)


3

Test Matrix $a$ is rank deficient (rank $r<m$). The rank 3 means there are 3 sets of dependent columns, each with 1 or more columns in it. The columns $0,1$ and $2,3$ are linearly dependent. Column 4 is linearly independent from all other columns. Finding independent columns will give a single representant of each set, while the choice is non-unique if any set has more than one element. If a column is independent, there can be other columns that are dependent on it but it can also be a single element in its set.

$$a\cdot x = t$$

In [4]:
x = np.array([1.0,2.0,3.0,4.0,5.0])  #param
t = a.dot(x)                         #result
t

array([ 15.00,  10.00,  23.00,  23.00,  10.00,  11.50])

## Sets of dependent columns

Assuming $m \times n$ Matrix $a$ with $m>=n$.

In [5]:
import scipy.linalg as sla
Qp,Rp,P = sla.qr(a, pivoting=True, check_finite=True)
print(Rp.diagonal())
print(P)
print()
Q,R = np.linalg.qr(a, mode='complete')
print(R.diagonal())

[-7.50 -4.90  1.87 -0.00 -0.00]
[3 1 4 0 2]

[-2.45  0.00 -1.50  0.00 -0.89]


Find columns that are linearly independent. For non-pivoting QR, the columns in R with diagonal elements that are not zero are independent. For pivoting QR this does not work. However, as there are as many independent columns as the rank of the matrix, taking the first rank columns of the permutation also gives the independent column indices but in different order. The choices are not the same but from the same sets.

In [6]:
tol = 1e-6
print(R.diagonal())
indepsQR = np.where(np.abs(R.diagonal()) > tol)[0]
print(indepsQR)

print(np.where(np.abs(Rp.diagonal()) > tol)[0].size)   #rank with pivoting QR
indepsQRP = P[0:r]
print(indepsQRP)


[-2.45  0.00 -1.50  0.00 -0.89]
[0 2 4]
3
[3 1 4]


In [7]:
#create proper permutation matrix
Pp = np.zeros((P.size, P.size))
for i in P:
    Pp[i,P[i]]=1

In [8]:
#Gautier 1990, 3-4: linear relationships
#W1W2 = a.dot(Pp)
#W1 = W1W2[:,0:indeps.size]
#W2 = W1W2[:,indeps.size:a.shape[1]]
#Q,R = np.linalg.qr(W1W2)
#print(Q)
#print(R.diagonal())
#corresponds to QR with pivoting
#print(Qp)
#print(Rp.diagonal())

#dependent columns W1 as linear combinations of independent columns W1
R1 = Rp[0:indepsQRP.size,0:indepsQRP.size]
R2 = Rp[0:indepsQRP.size,indepsQRP.size:]
linDeps = np.linalg.inv(R1).dot(R2)
print(linDeps)

[[ 0.00  0.20]
 [ 0.50  0.00]
 [ 0.00 -0.00]]


In [9]:
print(P)
for i in range(0,linDeps.shape[0]):
    for j in range(0,linDeps.shape[1]):
        if np.abs(linDeps[i,j]) > tol:
            orgColi = P[indepsQRP.size+i]
            orgColj = P[j]
            print(
                '''col {} in W2(col {} in a) is a linear combination of col {} in W1 (col {} in a)'''\
                   .format(i, orgColi, j, orgColj))

[3 1 4 0 2]
col 0 in W2(col 0 in a) is a linear combination of col 1 in W1 (col 1 in a)
col 1 in W2(col 2 in a) is a linear combination of col 0 in W1 (col 3 in a)


## Getting the base projection matrix

The first $r$ orthogonal columns of $Q$ form a basis $B$ of the column space of $A$ in case a is a square matrix. In the rectangular case, the QR decomposition of $A^T$ seems to be possible but maybe not always solvable?. 
$B$ can be used to project the matrix $A$ with regard to a vector $v$ to a matrix $A_\text{base}$ with regard to a vector $v_\text{base}$.

In [21]:
Qp,Rp,P = sla.qr(a.T, pivoting=True, mode='economic')
print(Qp)
B = Qp[:, 0:r]
print(B)
#B = sla.orth(a)[0:a.shape[1], :]
#print(B)

[[ 0.00 -0.45 -0.00  0.89  0.00]
 [-0.00 -0.89  0.00 -0.45 -0.00]
 [-0.20 -0.00 -0.00  0.00 -0.98]
 [-0.98 -0.00  0.00  0.00  0.20]
 [-0.00 -0.00 -1.00  0.00  0.00]]
[[ 0.00 -0.45 -0.00]
 [-0.00 -0.89  0.00]
 [-0.20 -0.00 -0.00]
 [-0.98 -0.00  0.00]
 [-0.00 -0.00 -1.00]]


$Pp^T XB = [XB1 ; XB2] = [X1 + X1^{-1}R2 X2;0]$

In [23]:
print(P)
Px = Pp.dot(x)
print('x:\t', x)
x1 = Px[0:r]
x2 = Px[r:]
print ('x1, x2:\t', x1, x2)
xb1 = x1 + np.linalg.inv(R1).dot(R2.dot(x2))
print (xb1)

[2 4 0 3 1 5]
x:	 [ 1.00  2.00  3.00  4.00  5.00]
x1, x2:	 [ 4.00  2.00  5.00] [ 1.00  3.00]
[ 4.60  2.50  5.00]


In [22]:
a_base = a.dot(B)
x_base = B.T.dot(x)
print('x_base:\t', x_base)
t_base = a_base.dot(x_base)
print('t_base:\t', t_base)
print('t:\t', t)
print ('x*:\t', B.dot(x_base))    #should give same x as in standard space
print('x:\t', x)

print(a.dot(np.eye(5) - B.dot(B.T)))

x_base:	 [-4.51 -2.24 -5.00]
t_base:	 [ 15.00  10.00  23.00  23.00  10.00  11.50]
t:	 [ 15.00  10.00  23.00  23.00  10.00  11.50]
x*:	 [ 1.00  2.00  0.88  4.42  5.00]
x:	 [ 1.00  2.00  3.00  4.00  5.00]
[[-0.00  0.00 -0.00 -0.00  0.00]
 [-0.00  0.00 -0.00 -0.00  0.00]
 [-0.00 -0.00 -0.00 -0.00  0.00]
 [-0.00 -0.00 -0.00 -0.00  0.00]
 [ 0.00  0.00 -0.00 -0.00  0.00]
 [-0.00 -0.00 -0.00 -0.00  0.00]]
